### Load Libraries

In [57]:
import requests
import pandas as pd
from time import sleep
from random import randint
from datetime import datetime
from datetime import timedelta

### Parameters

In [58]:
offset = 8
duration = 30
sleep_time = 3

In [59]:
param_details = {
            'PS':('Surface Pressure','surf_press'),
            'WS10M':('Wind Speed at 10 Meters','wnd_spd_10m'),
            'WS50M':('Wind Speed at 50 Meters','wnd_spd_50m'),
            'WD10M':('Wind Direction at 10 Meters','wnd_dir_10m'),
            'WD50M':('Wind Direction at 50 Meters','wnd_dir_50m'),
            'QV2M':('Specific Humidity at 2 Meters','spc_hmd_2m'),
            'RH2M':('Relative Humidity at 2 Meters','rel_hmd_2m'),
            'PRECIPITATIONCAL':('Precipitation','precip'),
            'T2M':('Temperature at 2 Meters','temp_2m'),
            'T2MDEW':('Dew/Frost Point at 2 Meters','dew_frst_2m'),
            'T2MWET':('Wet Bulb Temperature at 2 Meters','wet_temp_2m'),
            'TS':('Earth Skin Temperature','earth_temp')
            }

### Function Definitions

In [60]:
def bring_col_to_front(df,cols_to_reorder):
    original_columns=list(df.columns)
    cols_to_reorder.reverse()
    for reorder_col in cols_to_reorder:
        original_columns.insert(0, original_columns.pop(original_columns.index(reorder_col)))
    return df[original_columns]

In [61]:
def format_params(param_list):
    return '%2C'.join(param_list)

In [62]:
def rand_sleep(max_sec):
    sleep((randint(1,max_sec*1000))/1000)

In [63]:
def generate_url(start_date,end_date,latitude,longitude,parameters):
    return "https://power.larc.nasa.gov/api/temporal/daily/point?"\
                                                +"start="+str(start_date)\
                                                +"&end="+str(end_date)\
                                                +"&latitude="+str(latitude)\
                                                +"&longitude="+str(longitude)\
                                                +"&community=re&parameters="+parameters\
                                                +"&format=json&header=true&time-standard=utc"

In [64]:
def get_start_end_date(offset,duration):
    return (datetime.now() - timedelta(days = (offset+duration))).strftime("%Y%m%d"), (datetime.now() - timedelta(days = offset)).strftime("%Y%m%d")

In [65]:
def get_weather_info(latitude,longitude):
    start_date, end_date = get_start_end_date(offset,duration)
    parameters = format_params(list(param_details.keys()))
    url = generate_url(start_date,end_date,latitude,longitude,parameters)
    response = requests.get(url)
    if(response.status_code == 200):
        print("Fetching weather info for: ",latitude,",",longitude,"..")
        results = response.json()
        df_weather_info=pd.DataFrame.from_dict(results['properties']['parameter'], orient='index').T
        df_weather_info['latitude']=results['geometry']['coordinates'][1]
        df_weather_info['longitude']=results['geometry']['coordinates'][0]
        df_weather_info=bring_col_to_front(df_weather_info,['latitude','longitude'])
        df_weather_info=df_weather_info.reset_index(drop=True)
        df_mean_weather_info = pd.DataFrame(columns=['latitude',
                                     'longitude',
                                     'PS',
                                     'WS10M',
                                     'WS50M',
                                     'WD10M',
                                     'WD50M',
                                     'QV2M',
                                     'RH2M',
                                     'PRECIPITATIONCAL',
                                     'T2M',
                                     'T2MDEW',
                                     'T2MWET',
                                     'TS'])
        
        df_mean_weather_dict={}
        for col in df_weather_info.columns:
            df_mean_weather_dict[col]=[df_weather_info[col].mean()]
        return pd.DataFrame.from_dict(df_mean_weather_dict)
    else:
        print("Bad response. Check the URL again.")

### Main Execution Block

In [66]:
df_wndfrm_power_info=pd.read_csv("output_files/windfarms_power_stats.csv")

In [67]:
df_wndfrm_power_info = df_wndfrm_power_info[df_wndfrm_power_info['windfarm_power']>0]

In [ ]:
df_wndfrm_weather_info = pd.DataFrame(columns=['latitude',
                             'longitude',
                             'PS',
                             'WS10M',
                             'WS50M',
                             'WD10M',
                             'WD50M',
                             'QV2M',
                             'RH2M',
                             'PRECIPITATIONCAL',
                             'T2M',
                             'T2MDEW',
                             'T2MWET',
                             'TS'])

for index, row in df_wndfrm_power_info.iterrows():
    df_wndfrm_weather_info = pd.concat([df_wndfrm_weather_info,get_weather_info(row['latitude'],row['longitude'])],axis=0)
    rand_sleep(sleep_time)

In [70]:
df_wndfrm_dataset=pd.concat([df_wndfrm_weather_info.reset_index(drop=True),df_wndfrm_power_info.drop(['latitude','longitude','windfarm_name'],axis=1)],axis=1)
df_wndfrm_dataset.to_csv("output_files/df_wndfrm_dataset.csv",index=False)